# Prepare code and data.
Cloning repository and mounting google drive.

In [ ]:
!git clone https://github.com/Intelligent-Systems-Phystech/2022-Project-94

Cloning into '2022-Project-94'...
remote: Enumerating objects: 924, done.
remote: Counting objects: 100% (320/320), done.
remote: Compressing objects: 100% (241/241), done.
remote: Total 924 (delta 121), reused 273 (delta 79), pack-reused 604
Receiving objects: 100% (924/924), 1.96 GiB | 17.05 MiB/s, done.
Resolving deltas: 100% (317/317), done.


In [ ]:
import os
os.rename("2022-Project-94", "HailProject")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

# Import everything what is needed for experiment

In [ ]:
import sys
import os
import HailProject.code.src.data_processing as dp
from HailProject.code.src.utils import *
from scipy import signal
import numpy as np
from matplotlib import pyplot as plt
import torch
from HailProject.code.model.losses import EVL
from HailProject.code.model.HailNet import HailNet, train
from torch.optim import Adam, SGD
from sklearn.metrics import accuracy_score, classification_report
from HailProject.code.model.HailNet import test

# Vizualize the data

In [ ]:
feature_names = ['runoff']
numpys_26 = dp.get_nps(feature_names, "drive/MyDrive/Data/Hail/2017-6-29/*.tif") 
numpys_25 = dp.get_nps(feature_names, "drive/MyDrive/Data/No Hail/2017-5-10/*.tif") 
tmean_26 = np.nan_to_num(numpys_26['runoff'])
tmean_25 = np.nan_to_num(numpys_25['runoff'])
print("2006-5-8")
print("Number of dimensions: ", len(tmean_26.shape))
print("First dimension: time - ", tmean_26.shape[0], " - hours")
print("Second dimension: longtitude - ", tmean_26.shape[1], " - pixels (points)")
print("Third dimension: latitude - ", tmean_26.shape[2], " - pixels (points)")
print("2006-5-16")
print("Number of dimensions: ", len(tmean_25.shape))
print("First dimension: time - ", tmean_25.shape[0], " - hours")
print("Second dimension: longtitude - ", tmean_25.shape[1], " - pixels (points)")
print("Third dimension: latitude - ", tmean_25.shape[2], " - pixels (points)")
for i in range(12):
    plt.figure(figsize=(10, 5))
    plt.imshow(tmean_25[i], cmap='hot', interpolation='nearest')
    plt.show()
print("_______________________________________________________________")
for i in range(12):
    plt.figure(figsize=(10, 5))
    plt.imshow(tmean_26[i], cmap='hot', interpolation='nearest')
    plt.show()

# Training model


In [ ]:
num_epochs = 1 
n = 2
long = 359
lat = 533
gru_hidden_size = 16 * 16
gru_hidden_layers = 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HailNet(n, long, lat, gru_hidden_size, gru_hidden_layers)
loss_fn = torch.nn.BCELoss()
loss_fn.to(device)
model = model.to(device)
opt_adam = Adam(model.parameters(), lr=5e-4)
macroepoch = 10
for i in range(1, macroepoch + 1):

    print(f"\nMacro-Epoch: {i}\n")

    tdl, _ = dp.get_dataloader(feature_names = [
      'dewpoint_temperature_2m',
      'runoff'
      ],
      data_path="drive/MyDrive/Data",
      eco=True,
      part=i)
    
    losses_adam = train(num_epochs, model, loss_fn, opt_adam, tdl)


Macro-Epoch: 1

Epoch: 0 | Loss: 0.6812788248062134

Macro-Epoch: 2

Epoch: 0 | Loss: 0.4957490861415863

Macro-Epoch: 3

Epoch: 0 | Loss: 0.6858793497085571

Macro-Epoch: 4

Epoch: 0 | Loss: 0.33934760093688965

Macro-Epoch: 5

Epoch: 0 | Loss: 0.25601550936698914

Macro-Epoch: 6

Epoch: 0 | Loss: 0.19408318400382996

Macro-Epoch: 7

Epoch: 0 | Loss: 0.15692977607250214

Macro-Epoch: 8

Epoch: 0 | Loss: 0.664310097694397

Macro-Epoch: 9

Epoch: 0 | Loss: 0.6323258876800537

Macro-Epoch: 10

Epoch: 0 | Loss: 0.11481376737356186


# Testing model 

In [ ]:
predictions = []
true_values = []
macroepoch = 24
for i in range(11, macroepoch + 1):
    try:
      tdl_test, x1 = dp.get_dataloader(feature_names = [
        'dewpoint_temperature_2m',
        'runoff'
        ],
        data_path="drive/MyDrive/Data",
        eco=True,
        train=False,
        part=i,
        )
      
      preds, trues = test(model, tdl_test, ["accuracy"], {"accuracy": accuracy_score})
      predictions = predictions + preds
      true_values = true_values + trues 
    except NotImplementedError:
      pass

In [ ]:
predictions_ = torch.cat(predictions)
true_values_ = torch.cat(true_values)
threshold = 0.16
for i in range(predictions_.shape[0]):
    if predictions_[i] < threshold:
        predictions_[i] = 0
    else:
        predictions_[i] = 1
print(classification_report(np.array(true_values_.cpu()), predictions_.cpu().numpy().squeeze(), target_names=["No Hail", "Hail"]))

              precision    recall  f1-score   support

     No Hail       0.97      1.00      0.98       140
        Hail       1.00      0.64      0.78        14

    accuracy                           0.97       154
   macro avg       0.98      0.82      0.88       154
weighted avg       0.97      0.97      0.96       154

